# Heroes of Pymoli

In [1]:
import pandas as pd
import os

filepath=os.path.join("HeroesOfPymoli","purchase_data.json")
Original = pd.read_json(filepath)
Original.head()
#Original.count() Just checked that there weren't null values

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


## Player Count

In [2]:
players = len(Original["SN"].unique())

Summary = pd.DataFrame({"Total Player": [players]} )
Summary

,Total Player
0,573


## Purchasing Analysis (Total)

#### Number of Unique Items

In [3]:
unique_items = len(Original["Item ID"].unique())
avg_price= round(Original["Price"].mean(),2)
purchases = Original["Item Name"].count()
total_rev = Original["Price"].sum()

Summary = pd.DataFrame({"Number of Unique Items": [unique_items], 
                        "Average Price": [avg_price],
                        "Number of Purchases": [purchases],
                        "Total Revenue": [total_rev]})

Summary["Average Price"] =Summary["Average Price"].map("${:,.2f}".format)
Summary["Total Revenue"] = Summary["Total Revenue"].map("${:,.2f}".format)

Summary = Summary[["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"]]
Summary

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


In [4]:
# create a table with totals for unique players
unique = Original.drop_duplicates(subset="SN")

## Gender Demographics

In [5]:
genders = pd.DataFrame()
genders["Total Count"] = unique.groupby("Gender")["Price"].count()
genders["Percentage of Players"] = round(genders["Total Count"] / genders["Total Count"].sum() * 100,2)

genders = genders[["Percentage of Players","Total Count"]]
genders = genders.sort_values(by="Total Count", ascending=False)
genders

,Percentage of Players,Total Count
Gender,,
Male,81.15,465
Female,17.45,100
Other / Non-Disclosed,1.40,8


In [6]:
# # I did this and it works, but the previous input is much more concise.

# gender_counts = []
# gender_percent = []
# genders = Original["Gender"].unique()

# for gender in genders:
#     gender_df = Original[Original["Gender"]== gender]
#     gender_counts.append(len(gender_df["SN"].unique()))
#     gender_percent.append(round(len(gender_df["SN"].unique()) / players * 100,2))

# print(gender_counts)
# print(gender_percent)

# gender_demo_df = pd.DataFrame({"Genders": genders, "Percentage of Players": gender_percent, "Total Count": gender_counts})
# gender_demo_df

## Purchasing Analysis (Gender)

### Male

In [7]:
gender_purchase = pd.DataFrame()
gender_purchase["Purchase Count"] = Original.groupby("Gender")["Price"].count()
gender_purchase["Average Purchase Price"] = Original.groupby("Gender")["Price"].mean()
gender_purchase["Total Purchase Value"] = Original.groupby("Gender")["Price"].sum()
gender_purchase["Normalized Totals"] = gender_purchase["Total Purchase Value"] / unique.groupby("Gender")["Price"].count()

gender_purchase["Average Purchase Price"] = gender_purchase["Average Purchase Price"].map("${:,.2f}".format)
gender_purchase["Total Purchase Value"] = gender_purchase["Total Purchase Value"].map("${:,.2f}".format)
gender_purchase["Normalized Totals"] = gender_purchase["Normalized Totals"].map("${:,.2f}".format)

gender_purchase

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,"$1,867.68",$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


## Age Demographics

In [8]:
bins = [0,9,14,19,24,29,34,39,999]
labels = ["<10","10-14","15-19","20-24","25-29","30-34","35-29","40+"]
Original["Age Group"] = pd.cut(Original["Age"], bins, labels= labels)
unique = Original.drop_duplicates(subset="SN")

age_groups = pd.DataFrame()
age_groups["Total Count"] = unique.groupby("Age Group")["Age"].count()
age_groups["Percentage of Players"] = round(100 * age_groups["Total Count"] / age_groups["Total Count"].sum(),2)

age_groups = age_groups[["Percentage of Players","Total Count"]]
age_groups

,Percentage of Players,Total Count
Age Group,,
<10,3.32,19
10-14,4.01,23
15-19,17.45,100
20-24,45.20,259
25-29,15.18,87
30-34,8.20,47
35-29,4.71,27
40+,1.92,11


## Purchasing Analysis (Age)

In [9]:
age_purchase = pd.DataFrame()
age_purchase["Purchase Count"] = Original.groupby("Age Group")["Price"].count()
age_purchase["Average Purchase Price"] = Original.groupby("Age Group")["Price"].mean()
age_purchase["Total Purchase Value"] = Original.groupby("Age Group")["Price"].sum()
age_purchase["Normalized Totals"] = age_purchase["Total Purchase Value"] / unique.groupby("Age Group")["Price"].count()

age_purchase["Average Purchase Price"] = age_purchase["Average Purchase Price"].map("${:,.2f}".format)
age_purchase["Total Purchase Value"] = age_purchase["Total Purchase Value"].map("${:,.2f}".format)
age_purchase["Normalized Totals"] = age_purchase["Normalized Totals"].map("${:,.2f}".format)

age_purchase

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Group,,,,
<10,28,$2.98,$83.46,$4.39
10-14,35,$2.77,$96.95,$4.22
15-19,133,$2.91,$386.42,$3.86
20-24,336,$2.91,$978.77,$3.78
25-29,125,$2.96,$370.33,$4.26
30-34,64,$3.08,$197.25,$4.20
35-29,42,$2.84,$119.40,$4.42
40+,17,$3.16,$53.75,$4.89


## Top Spenders

In [27]:
spenders = pd.DataFrame()
spenders["Purchase Count"] = Original.groupby("SN")["Price"].count()
spenders["Average Purchase Price"] = Original.groupby("SN")["Price"].mean()
spenders["Total Purchase Value"] = Original.groupby("SN")["Price"].sum()
sorted_spenders = spenders.sort_values(by="Total Purchase Value", ascending=False)

sorted_spenders["Average Purchase Price"] = sorted_spenders["Average Purchase Price"].map("${:,.2f}".format)
sorted_spenders["Total Purchase Value"] = sorted_spenders["Total Purchase Value"].map("${:,.2f}".format)

sorted_spenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


## Most Popular Items

In [11]:
items = pd.DataFrame()
items["Purchase Count"] = Original.groupby(["Item ID","Item Name"])["Price"].count()
items["Total Purchase Value"] = Original.groupby(["Item ID","Item Name"])["Price"].sum()
items["Item Price"] = items["Total Purchase Value"] / items["Purchase Count"]

pop_items = items.sort_values(by="Purchase Count",ascending=False)

pop_items["Total Purchase Value"] = pop_items["Total Purchase Value"].map("${:,.2f}".format)
pop_items["Item Price"] = pop_items["Item Price"].map("${:,.2f}".format)

pop_items = items.sort_values(by="Purchase Count",ascending=False)
pop_items = pop_items[["Purchase Count","Item Price", "Total Purchase Value"]]
pop_items.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,2.35,25.85
84,Arcane Gem,11,2.23,24.53
31,Trickster,9,2.07,18.63
175,Woeful Adamantite Claymore,9,1.24,11.16
13,Serenity,9,1.49,13.41


## Most Profitable Items

In [12]:
profit_items = items.sort_values(by="Total Purchase Value",ascending=False)

profit_items["Total Purchase Value"] = profit_items["Total Purchase Value"].map("${:,.2f}".format)
profit_items["Item Price"] = profit_items["Item Price"].map("${:,.2f}".format)

profit_items = profit_items[["Purchase Count","Item Price","Total Purchase Value"]]
profit_items.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88


In [23]:
# Just some fun to see what brought the Other / non-disclosed avg purchase price up
Other = Original[Original["Gender"] =="Other / Non-Disclosed"]
Other_items = Other.groupby(["Item ID","Item Name"])["Price"].agg({"Total Revenue": sum})
Other_items.sort_values(by="Total Revenue", ascending=False)

C:\Users\Chris\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  app.launch_new_instance()


,,Total Revenue
Item ID,Item Name,
48,"Rage, Legacy of the Lone Victor",4.32
115,Spectral Diamond Doomblade,4.25
128,"Blazeguard, Reach of Eternity",4.00
61,Ragnarok,3.97
29,"Chaos, Ender of the End",3.79
155,War-Forged Gold Deflector,3.73
141,Persuasion,3.27
183,Dragon's Greatsword,2.36
157,"Spada, Etcher of Hatred",2.21
